# Data 200 - Grad Project

Sverre Rynning-Tønnesen, Isabel Slorer, Hans Erik Heum

## Structure


1) Introduction <br>
2) Data Sampling and Collection <br>
3) Data Cleaning <br>
4) Exploratory Data Analysis <br>
5) Feature Engineering <br>
6) Data Modeling and Inferences <br>
7) Conclusion <br>

### Introduction

Vet ikke helt hva vi skal ha med her enda, kanskje en forklaring på notebooken, evt samme som står i Abstract i rapporten

### Data Sampling and Collection

Her må vi skrive om hvordan vi har hentet dataen.
Nevne at vi henter ekstern data, og fra en annen oppgave. Hvorfor?

How were the data collected? <br>
Was there any potential bias introduced in the sampling process?


#____________Kommentarer: <br>
Skal vi vise head på hvert eneste dataframe vi laster inn??

In [ ]:
#Load all important libraries
import pandas as pd
import glob
import re
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from zipfile import ZipFile
import datetime

from sklearn import linear_model as lm
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge

import plotly as go
import plotly.graph_objects as go

import plotly.graph_objs as go
import pandas as pd
from plotly.offline import iplot, init_notebook_mode


In [ ]:

#initialize state_social_distancing_d
def init_state_social_distancing_actions(path):
  df_master = pd.DataFrame()
  flag = False
  files_in_folder = glob.glob(path)
  for filename in files_in_folder:    
    zip_file = ZipFile(filename)
    for text_file in zip_file.infolist():
      # if not text_file.filename.startswith('__MACOSX/'):
      if text_file.filename.endswith('.csv'):
        date = re.search('\d*-\d*-\d*', text_file.filename)[0]
        if date == "20201-06-01":
          date = "2021-06-01"
        date_time_value = pd.to_datetime(date)
        df = pd.read_csv(zip_file.open(text_file.filename), sep=",", header=0)
        df["Date"] = date_time_value
        df.rename(columns = {'Unnamed: 0':'State'}, inplace = True)
        df.drop((df[df.State.isin(["United States"])].index) | (df[df.State.isnull()].index), inplace=True)
      if not flag:
        df_master = df
        flag = True
      else:
        df_master = pd.concat([df_master, df])
  df_master.set_index(["Date", "State"], inplace=True)
  df_master.sort_index(inplace=True)
  return df_master

state_social_distancing_actions = init_state_social_distancing_actions(r'**csv_files/state_social_distancing_actions.zip')



#### Import vaccination data

Hvis man skal være helt nazi og konsis, så skal man kun ta 'CA' som Location for data cleaning

In [ ]:
def init_vaccination_df_from_zip(path, index_col):
    df_master = pd.read_csv(glob.glob(path)[0], compression='zip', header=0, sep=',', index_col=index_col)
    return df_master

vaccination_df = init_vaccination_df_from_zip(r'**csv_files/COVID-19_Vaccinations_in_the_United_States_Jurisdiction.csv.zip', ["Date"])
vaccination_df = vaccination_df[vaccination_df["Location"] == "CA"]
vaccination_df.index = pd.to_datetime(vaccination_df.index)
vaccination_df.sort_index(inplace=True)
vaccination_df

#### Import death rate data

In [ ]:
def init_daily_reports(path):
  df_master = pd.DataFrame()
  flag = False
  files_in_folder = glob.glob(path)
  for filename in files_in_folder:    
    zip_file = ZipFile(filename)
    for text_file in zip_file.infolist():
      # if not text_file.filename.startswith('__MACOSX/'):
      if text_file.filename.endswith('.csv'):
        date = re.search('\d*-\d*-\d*', text_file.filename)[0]
        date_time_value = pd.to_datetime(date)
        df = pd.read_csv(zip_file.open(text_file.filename), sep=",", header=0)
        df["Date"] = date_time_value
        df.rename(columns = {'Province_State':'State'}, inplace = True)
        df.drop((df[df.State.isin(["United States"])].index) | (df[df.State.isnull()].index), inplace=True)
      if not flag:
        df_master = df
        flag = True
      else:
        df_master = pd.concat([df_master, df])
  df_master.set_index(["Date", "State"], inplace=True)
  df_master.sort_index(inplace=True)
  return df_master

covid_daily_reports = init_daily_reports(r'**csv_files/csse_covid_19_daily_reports_us.zip')

#### Import infection data

In [ ]:
def init_infection_dataframe_from_zip(path, index_col):
    df_master = pd.read_csv(glob.glob(path)[0], compression='zip', header=0, sep=',', index_col=index_col)
    return df_master

infected_df = init_infection_dataframe_from_zip(r'**csv_files/United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv.zip', ["submission_date", "state"])

#### Import death counts by sex, age & state

In [ ]:
df_sex_age = pd.read_csv('csv_files/cdc_death_counts_by_sex_age_state.zip',header=0)
df_sex_age.head(5)


#### Import death counts by conditions

In [ ]:
df_condition = pd.read_csv('csv_files/cdc_death_counts_by_conditons.zip',header=0)
df_condition.head(5)


#### Import anxiety data

In [ ]:
df_anx_dep = pd.read_csv('csv_files/nchs_covid_indicators_of_anxiety_depression.zip',header=0)
df_anx_dep.head(5)


## 2. Data Cleaning

What type of data are you currently exploring? <br>
What is the granularity of the data? <br>
What does the distribution of the data look like? Are there any outliers? Are there any missing or invalid entries? <br>

In [ ]:
def clean_state_social_distancing_actions(df):
  df = df.drop(columns=["Primary Election Postponement"])
  return df

cleaned_state_social_distancing_actions = clean_state_social_distancing_actions(state_social_distancing_actions)
cleaned_state_social_distancing_actions

Clean up the dataframe to remove unused columns and solve for Nan fields. 
- Have to manually insert face mask requirements for recent months based on: https://statepolicies.com/data/graphs/face-masks/
- Face mask mandate was reintroduced from Dec 15, 2021 -> Feb 15, 2022

Other changes
- Manually inserted "No Limit" after 2021-08-15 because missing data

In [ ]:
#### Clean restriction data

In [ ]:
california_data = cleaned_state_social_distancing_actions[cleaned_state_social_distancing_actions.index.get_level_values('State').isin(['California'])]
california_data.reset_index("State", inplace=True)
california_data = california_data[~california_data.index.duplicated(keep='first')]
# Fill inn missing dates with rows equal the previous date with data
days_idx = pd.date_range(start=california_data.index[0], end="2022-04-18", freq="D")
california_data = california_data.reindex(days_idx, method="pad")
# Update facemask data
california_data.loc[: "2020-06-17", "Face Covering Requirement"] = 0 # No
california_data.loc["2020-06-18" : "2021-06-14", "Face Covering Requirement"] = 1 # Yes
california_data.loc["2021-06-15" : "2021-12-14", "Face Covering Requirement"] = 0
california_data.loc["2021-12-15" : "2022-02-14", "Face Covering Requirement"] = 1
california_data.loc["2022-02-15" : , "Face Covering Requirement"] = 0
# Manually insert gathering limit for missing values
california_data.loc["2021-08-16 ":, "Large Gatherings Ban"] = "No Limit"
# Transform Large Gatherings Ban to a sevearity of the rules (1: no restrictions, 5: All gatherings prohibited)
california_data["Large Gatherings Ban"].replace({'All Gatherings Prohibited': 5, '>50 Prohibited': 4, 'Expanded Limit to 25 or Fewer': 3, '>25 Prohibited': 3, '>10 Prohibited': 2, 'No Limit': 1}, inplace=True)
# Set the missing restaurant values to open
california_data[california_data["Restaurant Limits"].isna()]["Restaurant Limits"] = "Open"

ohe_restaurant_limits = pd.get_dummies(california_data["Restaurant Limits"], prefix='Restaurant Limits', columns = 'Restaurant Limits')

selected_ca_restrictions = pd.concat([california_data[["Face Covering Requirement", "Large Gatherings Ban"]], ohe_restaurant_limits], axis=1)

#selected_ca_restrictions = california_data[["Face Covering Requirement", "Large Gatherings Ban"]]
selected_ca_restrictions

#### Clean vaccination data

Add ekstra columns to have the same time-range as the other datasets
- Insert 0 as the number of vaccinated since no-one was vaccinated at the first date of the current df

In [ ]:
days_idx = pd.date_range(start="2020-06-04", end="2020-12-13", freq="D")
days_with_missing_data = vaccination_df.reindex(days_idx, fill_value=0)
days_with_missing_data["Location"] = "CA"
days_with_missing_data.index.names = ['Date']
vaccination_df = days_with_missing_data.append(vaccination_df)

In [ ]:
vaccination_df_administered = pd.DataFrame(vaccination_df["Administered"])
vaccination_df_administered.rename(columns={"Administered": "Total Vaccines Administered"}, inplace=True)
vaccination_df_administered

vaccination_df_administered["Daily vaccinations"] = vaccination_df_administered.diff(periods=1)
vaccination_df_administered = vaccination_df_administered[~vaccination_df_administered.index.duplicated(keep='first')]
vaccination_df_administered.loc["2020-06-04", "Daily vaccinations"] = 0
vaccination_df_administered[(vaccination_df_administered["Daily vaccinations"] == 0) | (vaccination_df_administered["Daily vaccinations"] < 0)]

# Becuase there is a mistake in the data on 2022-01-27 where the total number of vaccines suddenly drops by -1593072 we have to solve this
# and chose to do it with setting this value to 0
vaccination_df_administered.loc["2022-01-27", "Daily vaccinations"] = 0
vaccination_df_administered = vaccination_df_administered[["Daily vaccinations"]]
vaccination_df_administered

Convert Total Vaccines from incremental to day by day number of 

#### Clean daily covid deaths

In [ ]:
covid_daily_reports = covid_daily_reports.reset_index()
states = ['California']
covid_daily_reports = covid_daily_reports[covid_daily_reports["State"].isin(states) == True]
covid_daily_reports.set_index(["Date"], inplace=True)

In [ ]:
deaths_data = pd.DataFrame(covid_daily_reports["Deaths"])
deaths_data["Daily Deaths"] = deaths_data.diff(periods=1)
deaths_data = deaths_data[~deaths_data.index.duplicated(keep='first')]
deaths_data.drop(["Deaths"], axis=1, inplace=True)
# For all the values where the death count is less than 0 we set the value to be equal 0
deaths_data[deaths_data["Daily Deaths"] < 0] = 0

#### Clean infection rate data

In [ ]:

infected_df.index = infected_df.index.set_levels([pd.to_datetime(infected_df.index.levels[0]), infected_df.index.levels[1]])
infected_df_CA = infected_df[infected_df.index.get_level_values('state').isin(['CA'])]
infected_df_CA = infected_df_CA.reset_index()
infected_df_CA.set_index(["submission_date"], inplace=True)
infected_df_CA = infected_df_CA[["tot_cases"]].sort_index()



Make sure all dataframes share the same date-time range
- selected_ca_restrictions: 2020-06-04 - 2022-04-18	
- vaccination_df: 2020-06-04 - 2022-04-20
- Death data: 2020-04-12 - 2022-03-28

In [ ]:
merged_data = pd.concat([selected_ca_restrictions["2020-06-04" : "2022-03-28"], vaccination_df_administered["2020-06-04" : "2022-03-28"], infected_df_CA["2020-06-04" : "2022-03-28"], deaths_data["2020-06-04" : ]], axis=1)
merged_data.head(5)


#### Clean deaths by sex, age & state

In [ ]:
#Kommentere hvorfor vi fjerner dette
states = ['Alabama','Alaska','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','Florida','Georgia','Hawaii','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan','Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire','New Jersey','New Mexico','New York','North Carolina','North Dakota','Ohio','Oklahoma','Oregon','Pennsylvania','Rhode Island','South Carolina','South Dakota','Tennessee','Texas','Utah','Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming']
df_sex_age = df_sex_age[df_sex_age["State"].isin(states) == True]
df_sex_age = df_sex_age[df_sex_age["Group"]== 'By Month']

df_sex_age["End Date"] = pd.to_datetime(df_sex_age["End Date"])


#### Clean condition data

In [ ]:
df_condition = df_condition[df_condition["State"].isin(states) == True]


#### Clean anxiety data

In [ ]:
df_anx_dep2 = df_anx_dep[df_anx_dep['State']=='California']

merged_no_index = merged_data.copy().reset_index()
merged_no_index['index'] = pd.to_datetime(merged_no_index['index'])
restriction_mental = pd.DataFrame()
for index, row in df_anx_dep2.iterrows():
    first_date = pd.to_datetime(row['Time Period Start Date'])
    last_date = pd.to_datetime(row['Time Period End Date'])
    
    interval_rows = merged_no_index[(merged_no_index['index']>= first_date) & (merged_no_index['index']<= last_date)]
    interval_rows = pd.DataFrame(interval_rows.mean()).T
    row = pd.DataFrame(row).T
    interval_rows.rename(index={0:row.index[0]},inplace=True) 
    result = pd.concat([row,interval_rows], axis=1,join='inner')
    restriction_mental = pd.concat([restriction_mental,result],axis = 0)
    
restriction_mental= restriction_mental.fillna(0)


In [ ]:
#Her må vi diskutere hva som skal være i Feature Engineering og hva som som skal være i Cleaning

restriction_mental['Value'] = restriction_mental['Value'].astype(float)
restriction_mental = restriction_mental.groupby('Time Period End Date').mean()
restriction_mental.index = pd.to_datetime(restriction_mental.index)
restriction_mental = restriction_mental.sort_index()
restriction_mental = restriction_mental.drop(["Time Period",'Low CI','High CI'], axis=1)
restriction_mental.head()

## Exploratory Data Analysis

Her har vi ikke så mye enda, men her skal vi egentlig plotte ting som gjør at vi forstår dataen bedre. <br>
Altså ikke nødvendigvis plots som vi bruker for å forstå feks sammenhengen mellom restriksjoner og dødsfall, men mer som feks relationplots, outliers, etc - bedre forståelse av selve dataen.

#### EDA death by sex, age & state

In [ ]:
#To-Do
#Legge på tittel på begge nullplotsene, og legge de inn ved siden av hverandre



#sns.heatmap(df_condition.isnull(),yticklabels = False, cbar = False, cmap='viridis')
#As we can see on the plot below, not many of our columns (that we are currently using) contains a lot of null values.
#Except Covid 19-Deaths.. we should look further into that.

sns.heatmap(df_sex_age.isnull(),yticklabels = False, cbar = False, cmap='viridis')
#As we can see there are a LOT of null values in important columns..
#Before we decide to with them lets remove irrelevant rows.


#### Correlation

As we can see on the correlation plot, and also in the list of the most Daily Death's most correlated values, we can see that the columns adressing openings of restaurants and Face Covering Requirements are the most correlated with Daily Deaths. This makes them important features when it comes to predicting _Daily Deaths_. <br> <br>

Looking at the map of correlations we can see that _Face Covering Requirements_ and _Large Gathering Bans_ are heavily correlated. This makes sense when it comes to implementations of restrictions. Another noticable trait in the plot is the heavy negative correlation between _Large Gathering Bans_ and _number of total cases_. This can be interpreted as when the large gathering bans value raises (which determines how strict the gathering bans are) the total cases of covid gets reduced.


In [ ]:
merged_data['Face Covering Requirement'] = merged_data['Face Covering Requirement'].astype(int)

f, ax = plt.subplots(figsize=(8,10))
plt.title('Correlation between features in the dataset')
sns.heatmap(merged_data.corr(),linewidth=0.5,cmap="Blues", square=True,annot=True)

correlation = merged_data.corr()
print("Daily Death's most correlated features: \n")
print(abs(correlation['Daily Deaths']).sort_values(ascending = False), "\n")

To get a better overview of the most important features, we made a correlation plot directly with Daily Deaths.
We notice that it is usually more deaths when there are stricter restrictions. This makes sense considering why the government implements restrictions.

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3 ,figsize=(15,4))
var = 'Face Covering Requirement' 
data = pd.concat([merged_data['Daily Deaths'], merged_data[var]], axis=1)
data.plot.scatter(x=var, y='Daily Deaths', ax=ax1);
var = 'Restaurant Limits_Reopened to Dine-in Service with Capacity Limits'
data = pd.concat([merged_data['Daily Deaths'], merged_data[var]], axis=1)
data.plot.scatter(x=var, y='Daily Deaths',ax=ax2);
var = 'Restaurant Limits_Open'
data = pd.concat([merged_data['Daily Deaths'], merged_data[var]], axis=1)
data.plot.scatter(x=var, y='Daily Deaths',ax=ax3);

We did the same analysis on the mental health data merged with restrictions. 
The value the determines how the average mental health is in California is noticeable correlated with _Restaurant Limits_Reopened to Dine-in Service with Capacity Limits_ and _Large Gatherings Ban_. 

In [ ]:

f, ax = plt.subplots(figsize=(8,10))
plt.title('Correlation between features in the dataset')
sns.heatmap(restriction_mental.corr(),linewidth=0.5,cmap="Blues", square=True,annot=True)

correlation = restriction_mental.corr()
print("Mental Health most correlated features: \n")
print(abs(correlation['Value']).sort_values(ascending = False), "\n")


#### Outliers

**Kan argumentere for at dette skal flyttes til Data Cleaning, men vet at det er nokså vanlig å ha det her også.** <br>

To get a better understanding of the shape, variability and the center of our statistical data, we make several boxplots. We make a boxplot for the most important features, or the only ones that numerical for each retrieved dataset. <br>
In the first dataset there are no outliers for _Daily vaccinations_ or _tot_cases_. Looking at the _Daily Deaths_ however it is noticable many datapoints outside the maximum whisker (Q3+1.5*IQR). These datapoints would normally be interpreted as outliers and hence removed. Due to some domain knowledge and research regarding Covid-19 we understood that the number of Daily Deaths is a variable with high variance. For instance, the parameter would have suddent increases during the development of new contagious covid variants. Therefore, even though it most likely will infect our model and predctions, we decided not to interpret them as outliers and therefore keep the values. <br>

Looking at the boxplot for the _Value_ that determines levels of mental health, we see that there are no outliers. This is good for decreasing variance in our model and increasing statistical power. One reason for the absence of outliers is that we averaged our mental health data values for each group of anxiety-category. This will cause outliers to be less outstanding. <br>

Below we have plotted boxplots for the datasets containing Covid-19 related deaths based on sex, age and conditions. Many outliers are visible in these pots. For the same reason to include covid 19 outliers mentioned above, we chose to keep the outliers in these datasets as well. Outstanding covid 19 deaths are very interesting for our analysis to get a better understanding of what is causing these 'spikes'.

In [ ]:

fig, (ax1,ax2) = plt.subplots(1,2,figsize=(15,4))
ax1.title.set_text('Boxplots of params in Daily Covid Dataset')
ax2.title.set_text('Boxplot of Value in mental health Dataset')
merged_data.boxplot(column=['Daily vaccinations','tot_cases','Daily Deaths'],ax=ax1)
restriction_mental.boxplot(column=['Value'],ax=ax2)

fig, (ax1,ax2) = plt.subplots(1,2,figsize=(15,4))
ax1.title.set_text('Boxplots of params in Covid Deaths by age dataset')
ax2.title.set_text('Boxplot of Covid-19 Deaths in Condition Dataset')
df_sex_age.boxplot(column=['Total Deaths','Pneumonia Deaths','Influenza Deaths','Pneumonia, Influenza, or COVID-19 Deaths'],ax=ax1)
df_condition.boxplot(column=['COVID-19 Deaths'],ax=ax2)


## Feature Engineering

In [ ]:
merged_data["Daily vaccinations"] = np.log(merged_data["Daily vaccinations"] + 1).fillna(0)
merged_data["tot_cases"] = np.log(merged_data["tot_cases"] + 1).fillna(0)
merged_data

### Data Modeling and Inferences

1) Deaths in California by Sex, Age & Condition <br>
2) Plot of Mental Health Value in the US over the year <br>
3) Prediciton of Death Rates based on restrictions <br>
4) Prediciton of Mental Health based on restrictions and Death Rates <br>

#### Deaths in California by Sex, Age & Condition

In [ ]:
init_notebook_mode()

fig = go.Figure()

df_sex_age_grouped = pd.DataFrame(df_sex_age.groupby(['End Date','Age Group'])["COVID-19 Deaths"].sum())
df_sex_age_grouped = df_sex_age_grouped.reset_index()
#fig.add_trace(go.Scatter(grouped_df_age, x="End Date", y='COVID-19 Deaths', color="Age Group",line_group="Age Group"))
#region, geo_region in geo.groupby('Geographical region'):
    
for name, frame in df_sex_age_grouped.groupby('Age Group'):
    fig.add_scatter(x=frame['End Date'], y=frame['COVID-19 Deaths'], name=name, mode='lines')

fig.update_layout(title='Total Deaths in the USA by year and age',xaxis={'title':'Date'},yaxis={'title':'Number of Deaths'})
iplot(fig)



In [ ]:
fig = go.Figure()
df_sex_age_cali = df_sex_age[df_sex_age['State']=='California']
df_sex_age_grouped = pd.DataFrame(df_sex_age_cali.groupby(['End Date','Age Group'])["COVID-19 Deaths"].sum())
df_sex_age_grouped = df_sex_age_grouped.reset_index()
#display(grouped_df_age)
#fig.add_trace(go.Scatter(grouped_df_age, x="End Date", y='COVID-19 Deaths', color="Age Group",line_group="Age Group"))

#region, geo_region in geo.groupby('Geographical region'):
    
for name, frame in df_sex_age_grouped.groupby('Age Group'):
    fig.add_scatter(x=frame['End Date'], y=frame['COVID-19 Deaths'], name=name, mode='lines')

fig.update_layout(title='Total Deaths in California by year and age',xaxis={'title':'Date'},yaxis={'title':'Number of Deaths'})
iplot(fig)

In [ ]:
fig = go.Figure()
df_sex_age_cali = df_sex_age[df_sex_age['State']=='California']
df_sex_age_grouped = pd.DataFrame(df_sex_age_cali.groupby(['End Date','Sex'])["COVID-19 Deaths"].sum())
df_sex_age_grouped = df_sex_age_grouped.reset_index()
#display(grouped_df_age)
#fig.add_trace(go.Scatter(grouped_df_age, x="End Date", y='COVID-19 Deaths', color="Age Group",line_group="Age Group"))

#region, geo_region in geo.groupby('Geographical region'):
    
for name, frame in df_sex_age_grouped.groupby('Sex'):
    fig.add_scatter(x=frame['End Date'], y=frame['COVID-19 Deaths'], name=name, mode='lines')

fig.update_layout(title='Total Deaths in California by year and Sex',xaxis={'title':'Date'},yaxis={'title':'Number of Deaths'})
iplot(fig)

In [ ]:
init_notebook_mode()

fig = go.Figure()

grouped_df_condition = pd.DataFrame(df_condition.groupby(['End Date','Condition Group'])["COVID-19 Deaths"].sum())
grouped_df_condition = grouped_df_condition.reset_index()
#display(grouped_df_age)
#fig.add_trace(go.Scatter(grouped_df_age, x="End Date", y='COVID-19 Deaths', color="Age Group",line_group="Age Group"))

fig = go.Figure()
#region, geo_region in geo.groupby('Geographical region'):
    
for name, frame in grouped_df_condition.groupby('Condition Group'):
    fig.add_scatter(x=frame['End Date'], y=frame['COVID-19 Deaths'], name=name, mode='lines')
fig.update_layout(title='Total Deaths in California by year and Condition',xaxis={'title':'Date'},yaxis={'title':'Number of Deaths'})
iplot(fig)

#Kommentar til figuren og datasettet
#Vet ikke helt om en som dør av respiratory diseases blir påvirket av covid-death..



#### Average Mental Health in US over the year

In [ ]:
grouped = pd.DataFrame(df_anx_dep.groupby('Time Period End Date')['Value'].mean()).reset_index()
grouped['Time Period End Date'] = pd.to_datetime(grouped["Time Period End Date"])

grouped = grouped.sort_values(by ='Time Period End Date')
grouped

plt.plot(grouped['Time Period End Date'],grouped['Value'])
plt.title('Average Mental Health Value in US over the year')
plt.xlabel('year')
plt.ylabel('value')

#### Restrictions and Total Deaths

In [ ]:

#face_mask_ on = pd.to_datetime('2020-06-18')

#restriction_mental
merged_data_2 = merged_data.copy() 
merged_data_2['Face Change'] = merged_data_2['Face Covering Requirement'].diff()
merged_data_2['Restaurant Change'] = merged_data_2['Restaurant Limits_Open'].diff()

mask_on = merged_data_2[(merged_data_2['Face Change']==1) | (merged_data_2['Face Change']==-1) ]
restaurant_on = merged_data_2[(merged_data_2['Restaurant Change']==1) | (merged_data_2['Restaurant Change']==-1) ]
plt.figure(figsize = (18,8))
merged_data_2.reset_index(inplace=True)
sns.lineplot(x = 'index', y = 'Daily Deaths',data = merged_data_2)
my_label = "mask on"
my_label2 = "mask off"

for index, row in mask_on.iterrows():
    if row['Face Change'] ==1:
        plt.axvline(x = index, color = 'r',
                label = my_label)
        my_label = '_nolegend_'
    else :
        plt.axvline(x = index, color = 'g',
                label = my_label2)
        my_label2 = '_nolegend_'
        
my_label3 = "restaurant close"
my_label4 = "restarant open"
days = datetime.timedelta(3)
for index, row in restaurant_on.iterrows():
    if row['Restaurant Change'] ==1:
        plt.axvline(x = index-days, color = 'b',
                label = my_label3)
        my_label3 = '_nolegend_'
    else :
        plt.axvline(x = index, color = 'y',
                label = my_label4)
        my_label4 = '_nolegend_'
        
        
plt.legend()
plt.title('Covid deaths together with Restrictions')
plt.xlabel('Date')

In [ ]:
restriction_mental_2 = restriction_mental.copy() 

plt.figure(figsize = (18,8))
restriction_mental_2.reset_index(inplace=True)

sns.lineplot(x = 'Time Period End Date', y = 'Value',data = restriction_mental_2)

my_label = "mask on"
my_label2 = "mask off"

for index, row in mask_on.iterrows():
    if row['Face Change'] ==1:
        plt.axvline(x = index, color = 'r',
                label = my_label)
        my_label = '_nolegend_'
    else :
        plt.axvline(x = index, color = 'g',
                label = my_label2)
        my_label2 = '_nolegend_'

my_label3 = "restaurant close"
my_label4 = "restarant open"

for index, row in restaurant_on.iterrows():
    if row['Restaurant Change'] ==1:
        plt.axvline(x = index-days, color = 'b',
                label = my_label3)
        my_label = '_nolegend_'
    else :
        plt.axvline(x = index, color = 'y',
                label = my_label)
        my_label2 = '_nolegend_'
        
plt.legend()
plt.title('Mental Health together with Restrictions')
plt.xlabel('Date')


In [ ]:
current_val = 5
start_day = '2020-06-04'
last_date = None
dict_dates = {}

for index, row in merged_data.iterrows():
    if current_val != row['Large Gatherings Ban']:
        dict_dates[start_day] = [last_date,current_val]
        current_val = row['Large Gatherings Ban']
        start_day = index
    else:
        last_date = index
#Must add last value
dict_dates[start_day] = [last_date,current_val]
        
color_dict = {5: '#7FFF00',4:'#458B00',3:'yellow',2:'#FF7F00',1:'red',0:'blue'}
plt.figure(figsize = (18,8))
ax =sns.lineplot(x = 'Time Period End Date', y = 'Value',data = restriction_mental_2)
for key in dict_dates:
    color = color_dict[dict_dates[key][1]]
    ax.axvspan(key,dict_dates[key][0], color=color, alpha=0.5)


ax.grid(color='#79818f', alpha=0.6, linestyle='solid', linewidth=0.7, axis='x')
ax.grid(color='#2B2A27', alpha=0.6, linestyle='dashed', linewidth=0.7, axis='y')


plt.title('Mental Health together with Large Gatherings')
plt.xlabel('Date')

#### Prediction of Death Rates

In [ ]:
train, val = train_test_split(merged_data, test_size = 0.1, random_state = 42)
Y_train = train["Daily Deaths"]
X_train = train.drop(["Daily Deaths"], axis=1)

linear_model = lm.LinearRegression(fit_intercept=True)
linear_model.fit(X_train, Y_train)
y_prediction = linear_model.predict(X_train)
rmse = mean_squared_error(Y_train, y_prediction, squared=False)
rmse
plt.scatter(Y_train, Y_train-y_prediction, alpha=0.5);
rmse

#clf = svm.SVC(kernel='linear', C=1, random_state=42)
#scores = cross_val_score(clf, X_train, Y_train, cv=5)
#scores

In [ ]:
# Followed example: https://towardsdatascience.com/improve-linear-regression-for-time-series-forecasting-e36f3c3e3534
# https://github.com/cerlymarco/MEDIUM_NoteBook/blob/master/ModelTrees_TimeSeries/ModelTrees_TimeSeries.ipynb
# https://github.com/cerlymarco/MEDIUM_NoteBook

X_train_, X_test_, y_train_, y_test_ = train_test_split(
    merged_data.drop('Daily Deaths', axis=1), 
    merged_data['Daily Deaths'], 
    test_size=0.3, shuffle=False)

X_train_.shape, X_test_.shape

In [ ]:
### PLOT STORE DATA ###

y_train_.plot(label='train', figsize=(16,6))
y_test_.plot(label='test')
plt.title("State: {}".format("California")); plt.legend();

In [ ]:
#model = lm.LinearRegression(fit_intercept=True)

#model = GridSearchCV(estimator=Ridge(), param_grid={'alpha': [1, 3, 5, 10, 20]}, 
#                     scoring='neg_mean_squared_error', cv=2, refit=True)
model = lm.LinearRegression(fit_intercept=True)
model.fit(X_train_, y_train_)

pred_lr = pd.Series(model.predict(X_test_), index = y_test_.index)
pred_lr.plot(label='linear_regression')
y_test_.plot(label='true', figsize=(10,6));


#model.best_params_

# Total cases for the last 2 week for each day

In [ ]:
# Find number of days with deaths equal to 0
#zero_death_days = merged_data.loc[merged_data["Daily Deaths"] == 0]
#zero_death_days.reset_index(inplace=True)
#zero_death_days['weekday'] = zero_death_days['index'].dt.dayofweek # Monday=0, Sunday=6
#zero_death_days

## After not setting days with negative numbers equal to zero then there are 13 days equal to 0
## 6 of them is a sunday, 6 is a saturday, and 1 is a monday

#### Prediction of Mental Health

In [ ]:
X_train_, X_test_, y_train_, y_test_ = train_test_split(
    restriction_mental.drop('Value', axis=1), 
    restriction_mental['Value'], 
    test_size=0.3, shuffle=False)

model = lm.LinearRegression(fit_intercept=True)
model.fit(X_train_, y_train_)

#pred_lr = pd.Series(model.predict(X_test_), index = y_test_.index)
#pred_lr.plot(label='linear_regression')
#y_test_.plot(label='true', figsize=(10,6));

pred_lr = pd.Series(model.predict(X_train_), index = y_train_.index)
pred_lr.plot(label='linear_regression')
y_train_.plot(label='true', figsize=(10,6));